In [3]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [33]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    text = re.sub(r'<img.+?>(<br ?/?>)?', '', text)
    text = re.sub(r'^<br ?/?>', '', text)
    return text.strip()

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews").a.get('href')
    if '/http://' in url_normal:
        url_normal = 'http://' + url_normal.split('/http://')[-1]
    else:
        url_normal = 'https://' + url_normal.split('/https://')[-1]
    date = soup.find('p', id="newsDate").text[1:-1]
    #title_easy = soup.find('h1', class_="article-main__title")
    #title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

# for new web easy
def scrape_easy_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    url_easy = 'https://' + soup.find('meta', property="og:url").get('content').split('/https://')[-1]
    url_normal = soup.find('div', class_="link-to-normal").a.get('href')
    date = soup.find('p', class_="article-main__date").text[1:-1]
    title_easy = soup.find('h1', class_="article-main__title")
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', class_="article-main__body article-body")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', class_="article-main__body article-body").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

def make_date_normal_old(hmd,time):
    year = hmd[:4]
    month = hmd[4:6]
    day = hmd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"


# for old web normal
def scrape_normal_one(html, url):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('span', class_="contentTitle").text.strip()
    hmd_ = url.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_date_normal_old(hmd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url,
        'datePublished':date,
        'dateModified':""
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    with open('nolinknormal.txt') as f:
        nolink = f.read().split()
    return sorted(set(notyet) - set(nolink))

def get_link_nogenre(start=0):
    notyet = []
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    for dic in data:
        if dic['genre'] == []:
            notyet.append(dic['url'])
    return notyet  

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [36]:
htmls = get_link(0)
print(len(htmls))
htmls[:10]

1121


['http://www3.nhk.or.jp/news/html/20151022/k10010278681000.html',
 'http://www3.nhk.or.jp/news/html/20151103/k10010292821000.html',
 'http://www3.nhk.or.jp/news/html/20151107/k10010298041000.html',
 'http://www3.nhk.or.jp/news/html/20151108/k10010298051000.html',
 'http://www3.nhk.or.jp/news/html/20151108/k10010298411000.html',
 'http://www3.nhk.or.jp/news/html/20151109/k10010298901000.html',
 'http://www3.nhk.or.jp/news/html/20151109/k10010299191000.html',
 'http://www3.nhk.or.jp/news/html/20151109/k10010299481000.html',
 'http://www3.nhk.or.jp/news/html/20151110/k10010299961000.html',
 'http://www3.nhk.or.jp/news/html/20151110/k10010300521000.html']

In [29]:
htmls = get_link_nogenre(); htmls[:50]

[]

# scrape

In [4]:
driver = webdriver.Chrome()

In [37]:
for nhkurl in tqdm.tqdm(htmls[3:]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    try:
        dic = scrape_normal_one(html, nhkurl)
        js(dic)
    except:
        print(nhkurl)

37s/it]



 75%|███████▌  | 843/1118 [17:54:54<5:25:25, 71.00s/it]



 75%|███████▌  | 844/1118 [17:56:05<5:23:09, 70.76s/it]



 76%|███████▌  | 845/1118 [17:56:16<4:00:53, 52.94s/it]http://www3.nhk.or.jp/news/html/20160807/k10010625421000.html




 76%|███████▌  | 846/1118 [17:57:26<4:22:29, 57.90s/it]



 76%|███████▌  | 847/1118 [17:58:47<4:53:37, 65.01s/it]



 76%|███████▌  | 848/1118 [18:00:13<5:20:27, 71.21s/it]



 76%|███████▌  | 849/1118 [18:01:29<5:26:33, 72.84s/it]



 76%|███████▌  | 850/1118 [18:02:46<5:30:47, 74.06s/it]



 76%|███████▌  | 851/1118 [18:03:55<5:22:18, 72.43s/it]



 76%|███████▌  | 852/1118 [18:05:04<5:16:22, 71.36s/it]



 76%|███████▋  | 853/1118 [18:06:24<5:26:40, 73.96s/it]



 76%|███████▋  | 854/1118 [18:07:33<5:18:54, 72.48s/it]



 76%|███████▋  | 855/1118 [18:08:34<5:02:41, 69.05s/it]



 77%|███████▋  | 856/1118 [18:09:44<5:02:51, 69.36s/it]



 77%|███████▋  | 857/1118 [18:11:04<5:15:09, 72.45s/it]



 77%|███████▋  | 858/1118 [18:12:22<5:22:1

In [29]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape_normal_one(html, 'http://www3.nhk.or.jp/news/html/20150929/k10010252101000.html')
print(dic)
js(dic)

{'id': 'k10010252101000', 'title': '選手強化費 クレジットカードのポイントで寄付', 'article': 'クレジットカードのポイントをオリンピックを目指すトップ選手の強化費への寄付として募るというユニークな取り組みが、ＪＯＣ＝日本オリンピック委員会とクレジットカード大手の協力で２９日から始まりました。\n\nこの取り組みは、２０２０年東京オリンピックなどを見据え、選手強化の充実が求められるなか、ＪＯＣがオリンピックの公式スポンサーでクレジットカード大手のＶＩＳＡと協力して始めました。具体的には、ＶＩＳＡカードの会員がカードの利用に伴ってたまるポイントをオリンピックを目指すトップ選手を対象とした強化費としてＪＯＣに寄付することを選択します。カード会社は、ポイントを現金化してＪＯＣの口座に振り込み、ＪＯＣが来年のリオデジャネイロ大会に参加する２８の競技団体に配分するということです。配分の割合は通常の強化費配分の指針となる競技団体のランクなどに応じて決めて、配分した金額はホームページ上で公表するということです。期間は当面、２９日から来年８月末までの予定で、５年後の東京大会などに向けては、直接、ＪＯＣに寄付できる仕組みも検討するということです。取り組みに参加しているカード会社は、２９日現在、１４８社あり、こうした寄付の取り組みは世界のオリンピック委員会でも初めてだということです。\n\n\n\n会見に同席したＪＯＣの理事で陸上、ハンマー投げの４０歳、室伏広治選手は、「強化のサポートが十分ではない競技団体や選手がまだまだいるので、１人でも多くの選手がメダルを取れるように、また、東京大会への機運が高まるようにお願いしたい」と呼びかけたうえで、「自分自身、カードを利用してためたポイントが期限を迎えて消えてしまうことがたびたびある。消えてしまう前に寄付して、多くのアスリートや若い世代に行き渡るようにしたいですね」と話し会見場を和ませていました。また、競泳女子の１８歳、渡部香生子選手は、「競技を続ける環境が整わずに苦労している選手がいる。今回の取り組みで強化の支援はもちろん、若い選手たちの励みになれば」と話していました。', 'genre': ['スポーツ'], 'keywords': [], 'url': 'http://www3.nhk.o

In [91]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'"genre":(.+?),', html)

# clean category & keyword

In [25]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

6305
2641


In [43]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  6305


[('社会', 1976),
 ('国際', 1652),
 ('ビジネス', 974),
 ('科学・文化', 962),
 ('スポーツ', 810),
 ('政治', 640),
 ('暮らし', 475),
 ('地域', 395),
 ('気象・災害', 309)]

In [42]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [45]:
normal = pd.read_json('nhkwebeasy.json')
normal.id.value_counts()

k10010600041000    1
k10010435721000    1
k10011907121000    1
k10012221391000    1
k10010541451000    1
                  ..
k10010533141000    1
k10012219701000    1
k10010518271000    1
k10010536941000    1
k10012248381000    1
Name: id, Length: 2661, dtype: int64

# NHK web easy (new) ID k1001140020 ~

In [48]:
with open('tobescraped.txt') as f:
    urls = f.read().split()
print(len(urls))

ids = pd.read_json('nhkwebeasy.json')['id'].tolist()
urls = [url for url in urls if url.split('/')[-1].strip('.html') not in ids]
print(len(urls))

1989
1983


In [27]:
driver = webdriver.Chrome()

In [49]:
for url in urls[:100]:
    print(url, end='\r')
    driver.get(url)
    time.sleep(6)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(20)
    html = driver.page_source.encode('utf-8')
    dic = scrape_easy_one_new(html)
    js_e(dic)






  0%|          | 0/100 [00:00<?, ?it/s]




  1%|          | 1/100 [00:50<1:23:41, 50.72s/it]




  2%|▏         | 2/100 [01:34<1:19:21, 48.59s/it]




  3%|▎         | 3/100 [02:41<1:27:40, 54.23s/it]




  4%|▍         | 4/100 [03:30<1:24:14, 52.65s/it]




  5%|▌         | 5/100 [04:17<1:20:32, 50.87s/it]




  6%|▌         | 6/100 [04:56<1:13:55, 47.19s/it]




  7%|▋         | 7/100 [05:39<1:11:21, 46.04s/it]




  8%|▊         | 8/100 [06:28<1:11:51, 46.86s/it]




  9%|▉         | 9/100 [07:08<1:07:54, 44.77s/it]




 10%|█         | 10/100 [07:50<1:06:12, 44.14s/it]




 11%|█         | 11/100 [08:38<1:07:06, 45.25s/it]




 12%|█▏        | 12/100 [09:29<1:08:57, 47.02s/it]




 13%|█▎        | 13/100 [10:13<1:06:35, 45.92s/it]




 14%|█▍        | 14/100 [10:56<1:04:43, 45.15s/it]




 15%|█▌        | 15/100 [11:46<1:05:52, 46.50s/it]




 16%|█▌        | 16/100 [12:26<1:02:38, 44.74s/it]




 17%|█▋        | 17/100 [13:18<1:04:55, 46.93s/it]




 18%|█▊        | 18/100 [

NameError: name 'nhkurl' is not defined

In [43]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
dic

{'id': 'k10010055981000',
 'title_easy': '雨や曇りの日が多くて野菜の値段が高くなる',
 'title_easy_ruby': '<ruby>雨<rt>あめ</rt></ruby>や<ruby>曇<rt>くも</rt></ruby>りの<ruby>日<rt>ひ</rt></ruby>が<ruby>多<rt>おお</rt></ruby>くて<ruby>野菜<rt>やさい</rt></ruby>の<ruby>値段<rt>ねだん</rt></ruby>が<ruby>高<rt>たか</rt></ruby>くなる',
 'article_easy': '<org>気象庁</org>によると、今月は雨や曇りの日が多くなっています。\n４月上旬の日照時間（＝太陽が出ていた時間）は、四国でいつもの年の２４％、東海地方で２７％、近畿地方で３１％、関東地方などで３９％でした。１９６１年に調べ始めてから最も短くなりました。\n１日から２０日までに雨が降った量もいつもの年より多くて、２倍近く降った地方もありました。\n<org>東京都中央卸売市場</org>によると、晴れた日が少なくて雨の日が多かったため、多くの野菜の値段が高くなっています。今月１８日の白菜ときゅうりの値段はいつもの年の２倍以上でした。\n<org>農林水産省</org>は「市場に出る野菜の値段は、今月の終わりごろまで高いと思います」と話しています。',
 'article_easy_ruby': '<span class="colorC"><ruby>気象庁<rt>きしょうちょう</rt></ruby></span>によると、<ruby>今月<rt>こんげつ</rt></ruby>は<ruby>雨<rt>あめ</rt></ruby>や<ruby>曇<rt>くも</rt></ruby>りの<ruby>日<rt>ひ</rt></ruby>が<ruby>多<rt>おお</rt></ruby>くなっています。\n４<ruby>月<rt>がつ</rt></ruby><ruby>上旬<rt>じょうじゅん</rt></ruby>の<ruby>日照時間<rt>にっしょうじかん</rt></ruby>（＝<ruby>太陽<rt>たいよう</rt></ruby>が<ruby>出<rt>で</rt><

In [44]:
js_e(dic)

In [36]:
soup = BeautifulSoup(html)
soup.find('div', class_='link-to-normal')

<div class="link-to-normal" id="js-regular-news-wrapper">
<a class="btn" href="https://www3.nhk.or.jp/news/html/20200206/k10012274671000.html" id="js-regular-news" target="_blank"><ruby>普通<rt>ふつう</rt></ruby>のニュースを<ruby>読<rt>よ</rt></ruby>む</a>
</div>